In [1]:
import json
import numpy as np

In [2]:
import pandas as pd

In [5]:
import csv, os, sys

In [133]:
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
import nltk

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    #review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [195]:
def add_pid(dd_json):
    review_text = BeautifulSoup(dd_json['content']).get_text()
    rr = re.sub("[^a-zA-Z]"," ", review_text.lower())
    rr = rr.split(" ")
    
    s=len(rr)
    rr_new = rr[0:s/1]+[dd_json['item_id']]+rr[s/2:s]
    return rr_new

In [62]:
from collections import defaultdict
from collections import Counter
f =open("reviews_Grocery_and_Gourmet_Food.json",mode='r')
L = f.readlines()
review_list=[]
review_list_prod=[]
product_id = []
d=defaultdict(list)
review_dict_list=[]
c=0
for l in L:
    dd={}
    jj = json.loads(l)
    product_id+=[jj['asin']]
    #if jj["asin"] == "B00LQWKDBM":
    d[jj['asin']].append(jj['reviewText'])
    if (len(jj['reviewText'].split(" "))>12):
        review_list += [jj['reviewText']] 
    
    dd["item_id"] = jj['asin']
    dd["content"] = str(jj['reviewText'])
    c+=1
    dd_json=json.dumps(dd)
    review_dict_list+=[dd]
f.close()

In [196]:
all_text_pid = map(lambda x: add_pid(x),review_dict_list[0:100000])

In [194]:
all_text_pid[475]

u'wow  this arrived with two green rings    the same size     and no yellow ring   where is the quality control   didn t the packager at amazon notice this glaring mistake   what a disappointment   feels cheap and smells bad  pass on this one  B00000IRTW   didn t the packager at amazon notice this glaring mistake   what a disappointment   feels cheap and smells bad  pass on this one '

In [135]:
all_text_pid = map(lambda x: review_to_wordlist(x, tokenizer),all_text_pid)

In [139]:
import itertools
merged_text = list(itertools.chain(*all_text))
merged_text_pid = list(itertools.chain(*all_text_pid))

In [140]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

num_features = 300    # Word vector dimensionality                      
min_word_count = 1   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print "Training model..."
model = word2vec.Word2Vec(all_text_pid,
                          workers=num_workers,size=num_features, 
                          min_count = min_word_count, window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "rev_pid"
model.save(model_name)

Training model...


In [87]:
#model.vocab

In [215]:
model.most_similar(positive=['tea'])

[(u'B000E130DY', 0.7085045576095581),
 (u'B000CQBZPG', 0.7072957754135132),
 (u'teas', 0.7031727433204651),
 (u'B000CQBZQA', 0.6858943104743958),
 (u'B000CQBZOW', 0.645676851272583),
 (u'green', 0.6334189772605896),
 (u'B000CQIDBK', 0.629112720489502),
 (u'B000CQIDBU', 0.6187631487846375),
 (u'B000E1BMYI', 0.588849663734436),
 (u'B000CQBZQK', 0.588737428188324)]

In [198]:
model.most_similar(positive=['coffee'])

[(u'B000E1D7RS', 0.6467551589012146),
 (u'B000CMFKNS', 0.5814913511276245),
 (u'B000CEO6WM', 0.5619232654571533),
 (u'B000BTIHBO', 0.5466931462287903),
 (u'cup', 0.5308606028556824),
 (u'mahmet', 0.5111957788467407),
 (u'B000AAJQQO', 0.5107163190841675),
 (u'B000CDP1UY', 0.49537193775177),
 (u'comthere', 0.49242138862609863),
 (u'mehmet', 0.47717005014419556)]

In [151]:
model.most_similar(positive=['coffee'],topn=20)

[(u'B000E1D7RS', 0.6467551589012146),
 (u'B000CMFKNS', 0.5814913511276245),
 (u'B000CEO6WM', 0.5619232654571533),
 (u'B000BTIHBO', 0.5466931462287903),
 (u'cup', 0.5308606028556824),
 (u'mahmet', 0.5111957788467407),
 (u'B000AAJQQO', 0.5107163190841675),
 (u'B000CDP1UY', 0.49537193775177),
 (u'comthere', 0.49242138862609863),
 (u'mehmet', 0.47717005014419556),
 (u'coffees', 0.47468623518943787),
 (u'B000CEM4LW', 0.47376561164855957),
 (u'efendi', 0.47276073694229126),
 (u'B0009YJXTG', 0.47141870856285095),
 (u'B000BYALJK', 0.4704415202140808),
 (u'B000CL4GWU', 0.466859370470047),
 (u'B000C13ORS', 0.4631667137145996),
 (u'B000BN7VLW', 0.4608825445175171),
 (u'amman', 0.45444512367248535),
 (u'B000C119O8', 0.45338061451911926)]

In [199]:
model.most_similar(positive=['darjeeling'])

[(u'keemun', 0.715492844581604),
 (u'thathave', 0.7015607357025146),
 (u'ceylon', 0.6821877956390381),
 (u'castleton', 0.6731735467910767),
 (u'twining', 0.671531081199646),
 (u'assam', 0.6643950939178467),
 (u'quintessentially', 0.6603749394416809),
 (u'darjeelings', 0.6529108881950378),
 (u'unquestionably', 0.6443912386894226),
 (u'muscatel', 0.6426557898521423)]

In [200]:
model.most_similar(positive=["vanilla"],topn=20)

[(u'B000CPZSC8', 0.6581645011901855),
 (u'beans', 0.657178521156311),
 (u'B000C5UYF4', 0.6370257139205933),
 (u'B000COC0PC', 0.6277520656585693),
 (u'extract', 0.6196675300598145),
 (u'B000CR1ELU', 0.6186503171920776),
 (u'B000C5YVCQ', 0.612859845161438),
 (u'considerpremium', 0.6051232814788818),
 (u'B000CPYVGM', 0.5951237082481384),
 (u'B000CO81LE', 0.5776723623275757),
 (u'vanille', 0.5749683380126953),
 (u'B000CQ5D1I', 0.5708217024803162),
 (u'brul', 0.5670573711395264),
 (u'jeni', 0.5530774593353271),
 (u'flagrant', 0.5482066869735718),
 (u'maturation', 0.5448338985443115),
 (u'varitety', 0.5409057140350342),
 (u'vodkas', 0.5282551050186157),
 (u'myjar', 0.5264564156532288),
 (u'B000CQ44C2', 0.5244849920272827)]

In [201]:
model.most_similar(positive=["cheese"],topn=20)

[(u'B000CQ01GU', 0.669583261013031),
 (u'mac', 0.6677161455154419),
 (u'cheddar', 0.6188218593597412),
 (u'kraft', 0.6175388693809509),
 (u'B000CQ01LA', 0.6132234334945679),
 (u'B000CQ01NS', 0.5883665680885315),
 (u'macaroni', 0.5850459337234497),
 (u'annie', 0.5811018943786621),
 (u'annies', 0.542894721031189),
 (u'velveeta', 0.5375041961669922),
 (u'B000CQ25TQ', 0.5279425978660583),
 (u'cheesy', 0.5266759395599365),
 (u'balderson', 0.5236006379127502),
 (u'B000E199UC', 0.5178687572479248),
 (u'loyalists', 0.5172863006591797),
 (u'conesiur', 0.5125787854194641),
 (u'chessiest', 0.5082336068153381),
 (u'reggo', 0.5064014196395874),
 (u'chz', 0.5037295818328857),
 (u'schloch', 0.4992867708206177)]

In [160]:
model.most_similar(positive=["strawberry"],topn=20)

[(u'B000CORSA4', 0.5609015226364136),
 (u'theclif', 0.5580374002456665),
 (u'B000C2RTYG', 0.5568441152572632),
 (u'kiwi', 0.5384212136268616),
 (u'B000BJOEMA', 0.5196341276168823),
 (u'grape', 0.5030319690704346),
 (u'glico', 0.5027259588241577),
 (u'mangosteen', 0.4823492765426636),
 (u'sprarkling', 0.48038220405578613),
 (u'leathers', 0.4754270017147064),
 (u'lychee', 0.47245436906814575),
 (u'besttt', 0.4679737091064453),
 (u'clinches', 0.4674139618873596),
 (u'tarty', 0.4648945927619934),
 (u'crapshoot', 0.4592456817626953),
 (u'B0006HC75S', 0.4580153822898865),
 (u'uha', 0.4551580548286438),
 (u'preserves', 0.45375028252601624),
 (u'B000BKGNUA', 0.4472818970680237),
 (u'blk', 0.44720834493637085)]

In [216]:
model.most_similar(positive=["granola","cereal"],negative=['sugar'])

[(u'cereals', 0.5408421158790588),
 (u'B000CRIUNU', 0.47718480229377747),
 (u'B000AY9UKM', 0.4461808204650879),
 (u'kashi', 0.4441458284854889),
 (u'B000CS9ZW4', 0.43773049116134644),
 (u'B000AY9UNE', 0.43516474962234497),
 (u'B0007SNZBQ', 0.41788458824157715),
 (u'B000CSEFQ0', 0.41523095965385437),
 (u'B000CSCP1C', 0.4053013026714325),
 (u'kellogg', 0.3978545367717743)]

In [217]:
model.most_similar(positive=["citrus"])

[(u'sensha', 0.4757012724876404),
 (u'lemony', 0.4743439257144928),
 (u'B0001ENZJS', 0.47144025564193726),
 (u'B000BDGA5A', 0.46304553747177124),
 (u'tangerine', 0.45754915475845337),
 (u'cinnamony', 0.4547739028930664),
 (u'prominant', 0.4514232277870178),
 (u'verges', 0.44906294345855713),
 (u'tisane', 0.44839489459991455),
 (u'schizandra', 0.4471145272254944)]

In [218]:
model.most_similar(positive=['B0001ENZJS','B000BDGA5A'],negative=['sweet'],topn=20)

[(u'essn', 0.5945985317230225),
 (u'grapefruit', 0.5663565397262573),
 (u'tangerine', 0.5614561438560486),
 (u'B000BDHBWQ', 0.5542471408843994),
 (u'minneola', 0.5471541285514832),
 (u'B000BDHBWG', 0.5443358421325684),
 (u'B000BDDHPQ', 0.5294890999794006),
 (u'sanpelligrino', 0.5246742963790894),
 (u'citrus', 0.5217316150665283),
 (u'B000BH3K8G', 0.5174712538719177),
 (u'moet', 0.5151007175445557),
 (u'zinger', 0.5142519474029541),
 (u'giner', 0.5043891668319702),
 (u'verbena', 0.49788933992385864),
 (u'B000CHMJWI', 0.4945055842399597),
 (u'myrtle', 0.49374955892562866),
 (u'harmonious', 0.49080756306648254),
 (u'lime', 0.48945966362953186),
 (u'B000AXWA00', 0.48716098070144653),
 (u'icelotuszen', 0.4824545979499817)]

In [214]:
model.most_similar(positive=['hot','spicy'],topn=20)

[(u'clobbering', 0.4903775155544281),
 (u'B000CESZOM', 0.4875714182853699),
 (u'warm', 0.47784554958343506),
 (u'borring', 0.4697333574295044),
 (u'capp', 0.45062631368637085),
 (u'B000E148MG', 0.4487800896167755),
 (u'heat', 0.44477832317352295),
 (u'B000BUGFZ8', 0.43916523456573486),
 (u'B000A6HOVC', 0.4363606572151184),
 (u'chocopotle', 0.4272227883338928),
 (u'pepperyas', 0.423629492521286),
 (u'B000CMH0PY', 0.422413170337677),
 (u'bfs', 0.4189275801181793),
 (u'wisked', 0.4182887673377991),
 (u'B0002G3UWM', 0.418211430311203),
 (u'B000E123N2', 0.41808706521987915),
 (u'B000E123IC', 0.411756694316864),
 (u'B000CMGZFA', 0.40912503004074097),
 (u'B000A0WLFC', 0.4087831377983093),
 (u'turnoff', 0.40821367502212524)]

In [205]:
model.most_similar(positive=["licorice"])

[(u'egyptian', 0.6071937084197998),
 (u'anise', 0.5932295918464661),
 (u'dilatory', 0.5360476970672607),
 (u'B000CMF1A0', 0.5296076536178589),
 (u'licoriceyogi', 0.5259384512901306),
 (u'egiptian', 0.5249632596969604),
 (u'simiar', 0.5137002468109131),
 (u'ojibwa', 0.5058711767196655),
 (u'root', 0.5044517517089844),
 (u'fruitty', 0.490701287984848)]

In [206]:
model.most_similar(positive=["marmalade"])

[(u'B00061KXV4', 0.702656090259552),
 (u'dundee', 0.6739948987960815),
 (u'B0001IBP50', 0.6341570615768433),
 (u'keiller', 0.5858461856842041),
 (u'marmalades', 0.5578844547271729),
 (u'B000BJK16S', 0.556955099105835),
 (u'B00060ONFW', 0.5543650388717651),
 (u'bridges', 0.5532878637313843),
 (u'tawny', 0.5075708031654358),
 (u'sarabeth', 0.5040978193283081)]

In [207]:
model.most_similar('shipping',topn=20)

[(u'globalworx', 0.6046561002731323),
 (u'delivery', 0.5929408073425293),
 (u'prime', 0.539040207862854),
 (u'lollicup', 0.5311954021453857),
 (u'B000BT0LRC', 0.5265222191810608),
 (u'fee', 0.5202947854995728),
 (u'convinent', 0.5164339542388916),
 (u'trustworthiness', 0.5153446793556213),
 (u'subscibe', 0.5139398574829102),
 (u'charges', 0.511542022228241),
 (u'charged', 0.5024595260620117),
 (u'price', 0.5015949010848999),
 (u'B000ARTNM4', 0.4962100386619568),
 (u'shipped', 0.49488967657089233),
 (u'B000BIZT5M', 0.4945387542247772),
 (u'btls', 0.4918751120567322),
 (u'B000CEOXFW', 0.4917702078819275),
 (u'summaryitems', 0.491680383682251),
 (u'alwaysfind', 0.4900383949279785),
 (u'B000ASTPK8', 0.48250138759613037)]

In [147]:
coffee_words = model.most_similar(positive=["coffee"],negative=["tea"],topn=100)

In [150]:
f = open('word.tsv',mode='w')
for tt,ww in tea_words:
    f.write("%s\n"%tt)
f.close()

In [151]:
f = open('word_not_tea.tsv',mode='w')
for tt,ww in coffee_words:
    f.write("%s\n"%tt)
f.close()

In [148]:
tea_words=model.most_similar(positive=["tea"],negative=['coffee'],topn=100)

In [135]:
model.most_similar('shipping',topn=20)

[(u'prime', 0.6503046154975891),
 (u'charges', 0.6452329754829407),
 (u'charge', 0.6369796991348267),
 (u'handling', 0.6188427805900574),
 (u'eligible', 0.6038777828216553),
 (u'ships', 0.6022206544876099),
 (u'delivery', 0.6003583669662476),
 (u'ship', 0.5967106819152832),
 (u'shipped', 0.590171217918396),
 (u'fee', 0.5737879276275635),
 (u'charged', 0.5716139078140259),
 (u'membership', 0.5710250735282898),
 (u'tax', 0.5633741617202759),
 (u'postage', 0.5630922913551331),
 (u'discount', 0.5598807334899902),
 (u'coupon', 0.5554779767990112),
 (u'shipment', 0.5387856364250183),
 (u'costs', 0.5106035470962524),
 (u'pay', 0.5055798292160034),
 (u'outrageous', 0.505068838596344)]

In [136]:
model.most_similar('expected',topn=20)

[(u'described', 0.6036078333854675),
 (u'advertised', 0.5639945864677429),
 (u'expecting', 0.530653715133667),
 (u'exactly', 0.5173431634902954),
 (u'shape', 0.5100089311599731),
 (u'overall', 0.4958585798740387),
 (u'condition', 0.4882384240627289),
 (u'disappointed', 0.4674942195415497),
 (u'promised', 0.4632258415222168),
 (u'pleased', 0.45829206705093384),
 (u'expect', 0.4579944908618927),
 (u'earlier', 0.4548669457435608),
 (u'quickly', 0.45081350207328796),
 (u'nicely', 0.4434491991996765),
 (u'pictured', 0.43962106108665466),
 (u'fine', 0.43812114000320435),
 (u'arrived', 0.42969614267349243),
 (u'promptly', 0.4283469319343567),
 (u'ok', 0.42699915170669556),
 (u'disappointing', 0.42626938223838806)]

In [141]:
model.most_similar('fresh',topn=100)

[(u'plump', 0.48889070749282837),
 (u'moist', 0.4677954912185669),
 (u'packaged', 0.4577162265777588),
 (u'wonderful', 0.42614468932151794),
 (u'amazing', 0.4174938201904297),
 (u'sealed', 0.41142117977142334),
 (u'dried', 0.40930086374282837),
 (u'nice', 0.40703022480010986),
 (u'freshness', 0.4066725969314575),
 (u'vacuum', 0.40644919872283936),
 (u'used', 0.4037497639656067),
 (u'pliable', 0.40044116973876953),
 (u'quality', 0.39807724952697754),
 (u'packed', 0.3972757160663605),
 (u'coconut', 0.3893470764160156),
 (u'well', 0.38508063554763794),
 (u'way', 0.38409459590911865),
 (u'excellent', 0.3838505744934082),
 (u'soft', 0.37435460090637207),
 (u'fragrant', 0.3725663423538208),
 (u'fresher', 0.37251874804496765),
 (u'great', 0.36926260590553284),
 (u'use', 0.3692306876182556),
 (u'delicious', 0.369113564491272),
 (u'little', 0.368068128824234),
 (u'package', 0.36436712741851807),
 (u'smell', 0.363912969827652),
 (u'quickly', 0.3632248640060425),
 (u'bag', 0.36136025190353394),
 